In [1]:
import nibabel as nib
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

In [2]:
def save_ct_slices_as_images(input_path, output_folder, scan_name):
    """
    Saves each slice of a 3D CT scan from a .nii.gz file as individual .png images.

    Parameters:
        input_path (str): Path to the .nii.gz file.
        output_folder (str): Directory where the slices will be saved.
        scan_name (str): Base name for the output images (e.g., "nameOfScan").
    """
    # Load the .nii.gz file
    scan = nib.load(input_path)
    scan_data = scan.get_fdata()  # Get the 3D data as a NumPy array

    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through each slice along the Z-axis
    num_slices = scan_data.shape[2]
    for i in range(num_slices):
        slice_data = scan_data[:, :, i]  # Extract the i-th slice

        # Save the slice as a PNG file
        output_path = os.path.join(output_folder, f"{scan_name.split('.')[0]}_slice{i}.png")
        plt.imsave(output_path, slice_data, cmap="gray")  # Use grayscale colormap

    # print(f"Saved last slice as {output_path}")


def save_label_slices_as_images_no_grayscale(input_path, output_folder, scan_name):
    """
    Saves each slice of a 3D label map (e.g., segmentation labels) from a .nii.gz file as individual .png images
    without grayscale conversion. The label values are preserved.

    Parameters:
        input_path (str): Path to the .nii.gz label file.
        output_folder (str): Directory where the slices will be saved.
        scan_name (str): Base name for the output images (e.g., "nameOfLabels").
    """
    # Load the .nii.gz file
    scan = nib.load(input_path)
    scan_data = scan.get_fdata()  # Get the 3D data as a NumPy array

    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through each slice along the Z-axis
    num_slices = scan_data.shape[2]
    for i in range(num_slices):
        slice_data = scan_data[:, :, i]  # Extract the i-th slice

        # Convert the slice to uint8 for saving (if necessary)
        # If labels exceed 255, you may need to save as a different format.
        slice_uint8 = slice_data.astype(np.uint8)

        # Save the slice as a PNG file without grayscale conversion
        output_path = os.path.join(output_folder, f"{scan_name.split('.')[0]}_slice{i}.png")
        img = Image.fromarray(slice_uint8)
        img.save(output_path)

    # print(f"Saved last label slice as {output_path}")

In [ ]:
AMOS_PATH = r"amos22"

output_folders_normal = [
    os.path.join(AMOS_PATH, "Train/input"),
    os.path.join(AMOS_PATH, "Test/input"),
    os.path.join(AMOS_PATH, "Validation/input"),
]

output_folders_label = [
    os.path.join(AMOS_PATH, "Train/label"),
    os.path.join(AMOS_PATH, "Test/label"),
    os.path.join(AMOS_PATH, "Validation/label")
]

input_folders_normal = [
    os.path.join(AMOS_PATH, "imagesTr"),
    os.path.join(AMOS_PATH, "imagesTs"),
    os.path.join(AMOS_PATH, "imagesVa"),
]

input_folders_label = [
    os.path.join(AMOS_PATH, "labelsTr"),
    os.path.join(AMOS_PATH, "labelsTs"),
    os.path.join(AMOS_PATH, "labelsVa")
]

for folder in output_folders_normal:
    os.makedirs(folder, exist_ok=True)

for folder in output_folders_label:
    os.makedirs(folder, exist_ok=True)

#create input data
for output_folder, input_folder in zip(output_folders_normal, input_folders_normal):
    for file in tqdm(os.listdir(input_folder)):
        if not file.endswith(".nii.gz") or os.path.exists(os.path.join(output_folder, file)):
            continue
        save_ct_slices_as_images(os.path.join(input_folder, file), output_folder, file)

# create label data
for output_folder, input_folder in zip(output_folders_label, input_folders_label):
    for file in tqdm(os.listdir(input_folder)):
        if not file.endswith(".nii.gz") or os.path.exists(os.path.join(output_folder, file)):
            continue
        save_label_slices_as_images_no_grayscale(os.path.join(input_folder, file), output_folder, file)

  5%|▌         | 13/241 [01:51<32:01,  8.43s/it]

In [ ]:
#make sure there is no mismatch files

# Define directories
Train_image_dir = 'amos22/imagesTr'
Train_label_dir = 'amos22/labelsTr'

# Get list of file names in each directory
image_files = set(os.listdir(Train_image_dir))
label_files = set(os.listdir(Train_label_dir))

# Find files in imagesTr but not in labelsTr
extra_image_files = image_files - label_files

# Print the results
if extra_image_files:
    print("Files in 'imagesTr' but not in 'labelsTr':")
    for file in extra_image_files:
        print(file)
else:
    print("No extra files in 'imagesTr'.")